### Dependencies

In [58]:
import tstables
import tables as tb
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from datetime import *
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

### Store Data with TsTables

The set up of TsTables can be found in the `setup.py` file.

Open a pre-existing `clean_sp500_3yrs.h5ts` HDF5 file and get that timeseries from it.

In [2]:
h5 = tb.open_file('data/clean_sp500_3yrs.h5ts', 'r')
ts = h5.root.data._f_get_timeseries()

In [3]:
start = datetime(2023, 9, 25)
end = datetime(2023, 12, 23)

In [4]:
%time subset = ts.read_range(start, end)

CPU times: user 2.79 s, sys: 19.3 ms, total: 2.81 s
Wall time: 2.86 s


In [5]:
subset.head()

,A,AAPL,ABBV,ABNB,ABT,ACGL,ACN,ADBE,ADI,ADM,...,WY,WYNN,XEL,XOM,XYL,XYZ,YUM,ZBH,ZBRA,ZTS
2023-09-25,110.069542,174.372025,142.726974,134.139999,93.269531,77.165421,306.313019,511.600006,169.505753,71.964928,...,29.559948,88.728226,55.439793,108.374245,88.894646,45.590000,120.205078,115.037331,228.149994,173.857681
2023-09-26,108.879738,170.292007,142.044052,132.279999,92.082977,77.270020,300.147888,506.299988,166.520081,70.819817,...,29.248793,88.757584,53.790295,108.542068,87.458496,44.810001,119.281685,110.243713,223.960007,173.026123
2023-09-27,108.653580,168.776840,141.324188,134.029999,91.413139,77.098854,303.790985,502.600006,166.974228,71.089798,...,29.258221,90.255157,53.315022,112.075905,88.230301,44.139999,117.329094,108.577209,225.910004,170.883713
2023-09-28,110.128502,169.034332,140.512039,136.470001,93.891525,77.631355,290.639374,504.670013,170.452667,70.791893,...,28.767914,89.432961,52.998165,111.395256,88.953270,44.349998,119.349007,110.483109,236.869995,170.228287
2023-09-29,109.951515,169.549271,137.567947,137.210007,92.676254,75.796120,296.765839,509.899994,169.177231,70.214676,...,28.909346,90.450920,53.324337,109.632996,88.933723,44.259998,120.176224,110.247330,236.529999,170.198929


### Compute Asset Returns

In [6]:
# function to compute stock returns
def compute_returns(stock_price_array):
    stock_returns = (stock_price_array[1:, :] - stock_price_array[:-1, :]) / stock_price_array[:-1, :] * 100
    return stock_returns

In [7]:
%%time
start = datetime(2022, 10, 17)
end = datetime(2025, 10, 17) 
stock_price = ts.read_range(start, end)

CPU times: user 35.3 s, sys: 227 ms, total: 35.5 s
Wall time: 36.1 s


In [189]:
rows, columns = stock_price.shape
print('Total number of trading days over 3 years:', rows)
print('Total number of stocks:', columns)

Total number of trading days over 3 years: 754
Total number of stocks: 498


In [188]:
stock_price_array = stock_price.values
stock_returns = compute_returns(stock_price_array)
print('Daily returns of 498 S&P 500 stocks:')
print(stock_returns)
print('\nShape of daily stock returns:', stock_returns.shape)

Daily returns of 498 S&P 500 stocks:
[[ 1.33273175  0.94096123  0.13156765 ... -1.339248    0.8599602
   0.4205779 ]
 [-2.52455765  0.07653325 -1.01660172 ... -0.83462602 -1.29979272
  -1.522385  ]
 [-2.3418301  -0.3267145  -0.13975272 ... -1.65557368 -1.62404231
  -2.49781326]
 ...
 [ 1.1663291   0.63364896 -0.55826816 ...  1.02673018  1.50870827
  -0.93406631]
 [ 1.03831885 -0.75800089  0.28732822 ...  0.54336982  0.19771146
   0.80213864]
 [ 0.58114247  1.95594924  1.19011428 ...  2.00160132 -0.21093292
   0.55842739]]

Shape of daily stock returns: (753, 498)


### Compute Market Index Returns

In [10]:
%%time
index_df = pd.read_csv('data/index_sp500_3yrs.csv', index_col=0, parse_dates=True)

CPU times: user 4.05 ms, sys: 2.28 ms, total: 6.33 ms
Wall time: 73.5 ms


In [11]:
index_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 754 entries, 2022-10-17 to 2025-10-17
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Close   754 non-null    float64
 1   High    754 non-null    float64
 2   Low     754 non-null    float64
 3   Open    754 non-null    float64
 4   Volume  754 non-null    int64  
dtypes: float64(4), int64(1)
memory usage: 35.3 KB


In [12]:
index_price = index_df[['Close']]
index_price_array = index_price.values
index_returns = compute_returns(index_price_array)
print('\nShape of daily index returns:', index_returns.shape)


Shape of daily index returns: (753, 1)


### Compute Mean and Covariance of Stock Returns

In [100]:
mean_returns = np.mean(stock_returns, axis=0)
print('Mean returns of 498 S&P 500 stocks:')
print(mean_returns)
cov_returns = np.cov(stock_returns, rowvar=False)
print('\nVariance-covariance matrix of returns of 498 S&P 500 stocks:')
print(cov_returns)

Mean returns of 498 S&P 500 stocks:
[ 0.03064567  0.09288701  0.08674766  0.04241171  0.04507738  0.09748052
  0.00693667  0.04025822  0.10288137 -0.01121763  0.0410558   0.0755866
  0.06057457  0.06613101 -0.01565577  0.09613935  0.07334902  0.06348691
  0.07477449  0.00403491 -0.06480734 -0.00811097  0.07416883  0.1035352
  0.18537658 -0.01061968  0.23797958  0.07261434  0.0466541   0.09404864
  0.02612589  0.10522734  0.27250951  0.04126924  0.06077481 -0.02425857
  0.03279314  0.18993158  0.15401334  0.57469972  0.02287179 -0.03867943
  0.09229483  0.03222815  0.32582383  0.00897559  0.03053116  0.26363211
  0.13926006  0.08415863  0.08214039  0.08012631  0.01920993 -0.08139165
  0.07077524 -0.00717024  0.04022692 -0.08364345  0.04234958 -0.06985016
  0.15044428  0.15788222  0.11331716  0.13671531  0.11653459 -0.03518474
  0.07850445  0.08181207  0.0611322   0.12613253  0.11470849  0.04234702
  0.13748442 -0.05281065  0.12777446  0.08574711  0.16548933  0.05683381
  0.10504342  0.1

In [14]:
print('Shape of mean returns:', mean_returns.shape)
print('Shape of variance-covariance matrix of returns:', cov_returns.shape)

Shape of mean returns: (498,)
Shape of variance-covariance matrix of returns: (498, 498)


### Compute Beta

In [15]:
# stock_returns shape: (753, 498); index_returns shape: (753, 1)
cov_matrix = np.cov(stock_returns, index_returns, rowvar=False)
print('Shape of covariance matrix:', cov_matrix.shape)
cov_with_index = cov_matrix[:-1, -1]
index_var = np.var(index_returns, ddof=1)
beta = cov_with_index / index_var
print('\nBeta Coefficients:')
print(beta)
print('\nShape of Betas:', beta.shape)

Shape of covariance matrix: (499, 499)

Beta Coefficients:
[ 1.01425317  1.25407516  0.3256551   1.48133391  0.40043457  0.48740419
  0.93939264  1.22096727  1.50886827  0.45072658  0.6789169   1.28978582
  0.33644753  0.24585382  1.05655675  0.53768775  0.72792824  0.6834245
  0.4581012   0.97669271  1.68645363  1.55116986  0.52203593  0.89557427
  1.78054509  0.67339394  2.05494315  0.83404654  0.48330119  1.1298007
  0.36641071  1.46208373  1.64691243  0.47443023  0.80852288  1.25043287
  0.72683355  1.25597428  1.57560986  2.30028287  1.28094303  0.98273986
  0.35307095  0.733086    1.84084599  0.79204021  0.23998555  0.96606541
  1.21444701  0.34019187  1.01232524  0.98889872  0.76204944  0.744342
  1.19010246  0.51806818  1.1874252   0.59269083  0.46869905  0.64903127
  0.88411     0.99736156  0.97694347  1.37909805  1.14856733  0.28935799
  0.76070474  0.63319173  0.5387734   0.63505584  1.6651976   1.16531843
  1.17308813  0.13861503  0.3353812   1.1951863   1.09563082  0.34117

In [16]:
beta_df = pd.DataFrame({
    'Stock': stock_price.columns,
    'Beta': beta
})
print('Top 20 most volatile stocks:\n')
print(beta_df.sort_values(by='Beta', ascending=False).head(20))

Top 20 most volatile stocks:

    Stock      Beta
102  COIN  2.589684
39    APP  2.300283
314  MPWR  2.298602
223  HOOD  2.283811
444  TSLA  2.203025
369  PLTR  2.200885
340  NVDA  2.161722
406  SMCI  2.118144
26    AMD  2.054943
493   XYZ  2.027238
349    ON  2.017965
296  MCHP  1.970198
284  LRCX  1.913791
430   TER  1.881319
81    CCL  1.879553
324    MU  1.877238
44   AVGO  1.840846
447   TTD  1.805581
266  KLAC  1.794558
24   AMAT  1.780545


### PCA
We can think of PCA as a partitioning of the variation in the data

For now, we define feature/characteristic vector of each stock as its mean return + variance-covariance of its returns with other stocks.

In [17]:
feature_matrix = np.concatenate([mean_returns.reshape(len(mean_returns),1), cov_returns], axis=1)
print('Shape of the feature matrix:', feature_matrix.shape)

Shape of the feature matrix: (498, 499)


In [28]:
# standardize the data before applying PCA
scaler = StandardScaler()
X_scaled = scaler.fit_transform(feature_matrix)
X_scaled

array([[-0.56816116,  4.07560205,  0.11652581, ...,  1.14859476,
         0.48549676,  1.4181182 ],
       [ 0.20267571, -0.07455394,  3.51006995, ...,  0.03799078,
         0.33315256,  0.49509348],
       [ 0.12664206, -0.63500788, -1.31912409, ..., -0.1422172 ,
        -1.31268831, -0.77270279],
       ...,
       [-0.90219527, -0.36401781, -0.66133228, ...,  6.14602313,
        -0.51215899, -0.0433092 ],
       [-0.3445934 ,  1.9040603 ,  1.75780318, ...,  2.09046387,
         4.96117618,  2.50475575],
       [-0.78561676,  0.4514157 ,  0.05317914, ...,  0.88769983,
         0.07336679,  5.59256347]])

In [154]:
# create principal components
pca = PCA()
X_pca = pca.fit_transform(X_scaled)

# the max possible number of principal components equals the rank of X_scaled
component_labels = ['PC{}'.format(i+1) for i in range(pca.n_components_)]
X_pca = pd.DataFrame(X_pca, columns=component_labels)
print('Shape:', X_pca.shape, '\n')
X_pca.head()

Shape: (498, 498) 



,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,...,PC489,PC490,PC491,PC492,PC493,PC494,PC495,PC496,PC497,PC498
0,7.462692,5.145752,-4.372576,-6.231399,-2.820910,3.988588,2.700770,-1.212794,0.357487,-2.634799,...,-0.005384,-0.005993,-0.001348,0.004200,-0.001629,0.000545,0.000730,-0.000096,0.000021,8.693584e-17
1,3.946236,-4.451785,-4.752095,0.255295,-2.574090,-1.795662,1.005414,-0.065592,-2.976174,0.069296,...,-0.000113,-0.001572,0.004407,-0.001159,-0.004665,0.002067,-0.000829,-0.000344,-0.000037,8.693584e-17
2,-22.887418,-1.548461,0.211404,-5.945414,-1.382527,2.139143,2.864431,-3.510704,-0.029391,-3.201907,...,-0.001566,-0.001248,0.000104,0.001412,0.001990,0.001335,-0.000244,0.000240,-0.000019,8.693584e-17
3,20.953732,-4.780004,-3.699366,-1.252222,-0.525717,-5.683780,0.790201,0.211890,-0.358636,1.827893,...,-0.001896,-0.001471,-0.001630,0.001018,-0.001051,0.000032,-0.000018,-0.000038,0.000012,8.693584e-17
4,-19.777407,0.801989,-4.543959,0.047493,-0.988881,-1.207814,3.573193,-1.619958,-0.608275,-1.982072,...,-0.002272,-0.000918,-0.000899,-0.002312,0.001556,0.000343,-0.000129,-0.000502,0.000027,8.693584e-17


In [186]:
evr = pca.explained_variance_ratio_
evr_df = pd.DataFrame(evr, index=component_labels, columns=['Explained Variance Ratio'])
evr_df.head(30)

,Explained Variance Ratio
PC1,0.586919
PC2,0.132463
PC3,0.056109
PC4,0.040709
PC5,0.028340
PC6,0.012739
PC7,0.010389
PC8,0.008584
PC9,0.007286
PC10,0.005789


In [187]:
cv = np.cumsum(evr)
cv_df = pd.DataFrame(cv, index=np.arange(1, len(component_labels)+1), columns=['Cumulative Variance'])
cv_df.head(30)

,Cumulative Variance
1,0.586919
2,0.719382
3,0.775491
4,0.816200
5,0.844539
6,0.857279
7,0.867667
8,0.876252
9,0.883537
10,0.889326


Note that PCA depends entirely on the quality of the features we pass in.  
If our initial features characterize the data well, then PCA will find new orthogonal directions (principal components) that maximize variance.  
From the results above, it seems that our features are high-quality inputs. This is because:
1. They are economically meaningful (reflecting return and covariance structure)
2. The first 20 principal components explain over 92% of the total variance of the data

Therefore, we decide to use the top 20 principal components as our latent features, effectively reducing the original 499-dimensional feature space to 20 dimensions.

# K-Means Clustering
N = size of the stock universe  
p = number of features  
k = number of clusters = portfolio size